In [ ]:
# !pip install -U transformers

In [ ]:
# pip install --upgrade huggingface_hub

In [ ]:
# enter your own ngrok domain + auth token + hugging face "meta-llama/Meta-Llama-3-8B-Instruct" auth token




# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_domain = user_secrets.get_secret("domain")
# secret_value = user_secrets.get_secret("medhelp")
# secret_value_ngrok = user_secrets.get_secret("Ngrokk")


In [3]:
from huggingface_hub import login
login(token=secret_value)

In [ ]:
# from huggingface_hub import whoami

# user_info = whoami()
# print("Logged in as:", user_info["name"])

In [ ]:
import transformers
import torch

#  "meta-llama/Meta-Llama-3-8B-Instruct"



pipeline = transformers.pipeline(
    "text-generation",
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)


2025-07-18 10:13:49.968105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752833629.995000     197 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752833630.003132     197 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [5]:
chat_history = [
    {
        "role": "system",
        "content": """
You are a reliable, friendly healthcare assistant. You must ONLY provide general health-related information and tips — no diagnoses, no emergency advice, and no prescription guidance. Your role is to assist users safely and clearly, using their personal health data, and reply **strictly in JSON format**.

You are provided a JSON user profile with:
- age
- gender
- medical conditions
- medications

Instructions for every response:
1. Be **personalized**: Tailor suggestions based on age, gender, existing conditions, and medications.
2. Be **brief** and **clear**: Use simple, accurate language in under 100 words.
3. Be **safe**: Never suggest a diagnosis or treatment plan. If the query implies a need for diagnosis, add: _"It’s best to consult a doctor for proper medical evaluation."_
4. Be **polite**: If the query is off-topic or unclear, reply with: _"Please ask a clear health-related question so I can assist you better."_
5. Be **context-aware**: Don’t repeat tips already given. Avoid contradictions with their known health profile.
6. Always maintain a **warm, supportive** tone.
EXAMPLE RESPONSE:
{
  "reply": "After a walk, a light snack with protein and fiber—like a boiled egg with whole-grain toast—can help. Since you have diabetes, avoid sugary snacks. Also, stay hydrated!"
}

Reply only in a JSON object:
{
  "reply": "your response text here"
}
"""
    }
]


Since Kaggle Notebooks don’t support web servers directly via Flask (like app.run()), we’ll use a trick: run Flask using ngrok

In [8]:
!pip install flask
!pip install pyngrok

In [12]:
pip install flask-cors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{
  "reply": "Yes, you can go for a walk after taking your medicines! Since you have diabetes and hypertension, it's essential to monitor your blood sugar and blood pressure levels during and after the walk. Take your usual medications as prescribed and stay hydrated."
}


In [6]:

from flask_cors import CORS


from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
import time
import json
# Flask app
app = Flask(__name__)
CORS(app)
@app.route("/")
def index():
    return "Your health assistant is running on a static .app domain!"

@app.route("/healthbot", methods=["POST"])
def healthbot():
    data = request.get_json()
    query = data.get("query")

    user_age = query["age"]
    user_gender = query["gender"]
    user_diseases = query["diseases"]
    user_medication = query["medication"]
    user_input = query["input"]

    user_dict= dict(age=user_age,gender=user_gender,disease=user_diseases,medications=user_medication,input=user_input)
    json_string = json.dumps(user_dict)
    
    chat_history.append({"role": "user", "content": user_dict})
    # Define terminators
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    # Get response
    outputs = pipeline(
        chat_history,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.85,
        top_p=0.9,
        top_k=40,
    )
    
    # Parse assistant reply
    reply = outputs[0]["generated_text"][-1]["content"]
    chat_history.append({"role": "assistant", "content": reply})

    print(reply)
    return jsonify({"reply": reply})

def run_flask():
    app.run(port=5000)

# Start Flask server in background
thread = threading.Thread(target=run_flask)
thread.daemon = True
thread.start()

# Give Flask some time to start
time.sleep(2)

# Now start ngrok (after Flask is running)
ngrok.set_auth_token(secret_value_ngrok)
custom_domain = secret_value_domain  # Your .app domain

public_url = ngrok.connect(
    addr=5000,
    domain=custom_domain
)

print("Static Public URL:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Static Public URL: NgrokTunnel: "https://hamster-stirred-bluejay.ngrok-free.app" -> "http://localhost:5000"


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{
  "reply": "As a person with diabetes, it's essential to monitor your carbohydrate intake. Sweet corn is relatively high in carbs, so consider choosing a serving size of about 1/2 cup cooked. Also, pair it with a healthy protein source like beans or lean meat to balance your meal."
}


...

{
  "reply": "As a person with diabetes, here are some general health tips: Get at least 7-8 hours of sleep each night to help regulate your blood sugar levels. Consider incorporating stress-reducing activities like meditation, deep breathing, or yoga to manage stress. Wear sunscreen with at least SPF 30 and reapply every 2 hours to protect your skin from the sun. Avoid sharing personal care items, like razors or lip balm, to reduce the risk of infection. And remember to schedule regular health check-ups to stay on top of your health!"
}


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{
  "reply": "As a person with diabetes, it's generally recommended to monitor your carbohydrate intake. Bananas are a good source of natural sugars, and curd contains protein and calcium. When combined, they can provide a balanced snack. However, consider the portion size and choose a ripe banana to reduce the glycemic index. Also, be mindful of your overall carbohydrate intake and adjust your medication or meal plan as advised by your doctor."
}


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{
  "reply": "As a person with diabetes, it's essential to focus on balanced and nutrient-dense meals. For lunch, consider a meal that includes a source of protein, healthy fats, and complex carbohydrates. Some examples might include: grilled chicken or fish with brown rice and steamed vegetables, or a whole-grain pita with hummus, avocado, and mixed greens. Avoid or limit sugary drinks, processed meats, and foods high in saturated fats. Also, be mindful of portion sizes and adjust your medication or meal plan as advised by your doctor."
}


In [ ]:
# import requests
# print(requests.get("https://huggingface.co").status_code)